In [1]:
import os
import os.path as osp

import cv2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [11]:
def recover_image(img, t, A=np.array([1,1,1])):
    # set the minimum for the transmission map
    t_min = 0.1
    
    # the output dehazed images
    J = np.zeros(img.shape, dtype='float64')

    # iterate through all the pixels
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            # for every pixel invert the atmospheric scattering model
            # using the computed t-map and atmospheric light
            J[i,j] = ((img[i,j] - A) / max(t[i,j], t_min)) + A
            
    return J

In [17]:
def haze2dehazed(hazy_paths, tmap_paths, output_dir):
    
    # create the output directory if not already present
    os.makedirs(output_dir, exist_ok=True)
    
    # iterate through pairs of (hazy_image, t-map)
    for img_path, tmap_path in tqdm(zip(hazy_paths, tmap_paths), total=len(hazy_paths)):
        
        img_name = os.path.split(img_path)[1]
        
        dehazed_img_name = os.path.join(output_dir, img_name.replace("hazy", "dehazed"))
           
        img = cv2.imread(img_path)/255.
        tmap = cv2.imread(tmap_path)/255.
        tmap = cv2.cvtColor(tmap.astype('float32'), cv2.COLOR_BGR2GRAY)
        
        dehazed_img = recover_image(img, tmap)
    
        # write to disk
        cv2.imwrite(dehazed_img_name, dehazed_img*255)


In [18]:
dataset = "dh/NYU"

tmap_dir = osp.join("../datasets", dataset, "tmap_pred_refined")
hazy_dir = osp.join("../datasets", dataset, "hazy")
output_dir = osp.join("../datasets", dataset, "dehazed")

In [19]:
tmap_paths = []
hazy_paths = []

for f in sorted(os.listdir(hazy_dir)):
    hazy_path = os.path.join(hazy_dir, f)
    tmap_path = os.path.join(tmap_dir, f.replace("_hazy", "_tmap"))
    
    assert osp.isfile(tmap_path) and osp.isfile(hazy_path)
    tmap_paths.append(tmap_path)
    hazy_paths.append(hazy_path)  

In [20]:
print(hazy_paths[:5], tmap_paths[:5])

['datasets/dh/NYU/hazy/1000_hazy.bmp', 'datasets/dh/NYU/hazy/1001_hazy.bmp', 'datasets/dh/NYU/hazy/1002_hazy.bmp', 'datasets/dh/NYU/hazy/1003_hazy.bmp', 'datasets/dh/NYU/hazy/1004_hazy.bmp'] ['datasets/dh/NYU/tmap_pred_refined/1000_tmap.bmp', 'datasets/dh/NYU/tmap_pred_refined/1001_tmap.bmp', 'datasets/dh/NYU/tmap_pred_refined/1002_tmap.bmp', 'datasets/dh/NYU/tmap_pred_refined/1003_tmap.bmp', 'datasets/dh/NYU/tmap_pred_refined/1004_tmap.bmp']


In [ ]:
haze2dehazed(hazy_paths, tmap_paths, output_dir)

  2%|▏         | 35/1449 [01:10<46:55,  1.99s/it]